<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/adapters_2nd_try_classification_from_pretraining_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining

# Things to change in real run
* Use the proper datasets (not dev for validation?)




# Setup

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/AdaptOrDie'

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

In [ ]:
!pip install pynvml

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

In [ ]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_helpfulness"
train_dataset = load_dataset(dataset_name, split="train[:5%]")
train_2_dataset = load_dataset(dataset_name, split="train[5%:10%]")
validation_dataset = load_dataset(dataset_name, split="dev")

print(train_dataset)
print(train_2_dataset)
print(validation_dataset)

print_gpu_utilization()

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
print_gpu_utilization()

In [ ]:
adapter_hub_name = "adapter_classifier_from_prtrained_test"

# Load without head, we are adding one in the
adapter_name = model.load_adapter("BigTMiami/domain_adapter_test", with_head=False)


In [ ]:
# Add head for masked language modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
  )

# Set the adapter to be used for training
model.train_adapter(adapter_name)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Train

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="adapter_classifier_training_output",
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=25,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

In [ ]:
# No Eval before any training with new head

# eval_results = trainer.evaluate()
# non_trained_eval_loss = eval_results["eval_loss"]
# print_gpu_utilization()
# print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
# print(eval_results)

In [ ]:
results = trainer.train()
print_gpu_utilization()
print(results)

In [ ]:
eval_results = trainer.evaluate()
trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Trained Eval Loss: {trained_eval_loss:.2f}")
print(eval_results)

In [ ]:
training_args = TrainingArguments(
    output_dir="adapter_classifier_training_output",
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_2_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
results = trainer.train()
print_gpu_utilization()
print(results)

In [ ]:
eval_results = trainer.evaluate()
trained_eval_loss = eval_results["eval_loss"]
eval_f1_macro = eval_results["eval_f1_macro"]
print_gpu_utilization()
print(f"Trained eval_f1_macro:{100.0 * eval_f1_macro:.2f} Eval Loss: {trained_eval_loss:.2f}")
print(eval_results)

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()